In [7]:
import json

In [8]:
# data = json.load(open('cts_0301_0403_news.json', 'r', encoding='utf-8'))
file = open("cts_0301_0403_news.json", 'r', encoding='utf-8')
papers = []
for line in file.readlines():
    dic = json.loads(line)
    papers.append(dic)
type(papers)

list

# 初步斷詞

In [3]:
data_list = []
for i in papers:
    data_list.append(i["title"])
    data_list.append(i["content"])

In [3]:
data = str()
for i in data_list:
    data = str(data) + str(i)

In [4]:
print(type(data_list))
print(type(data))

<class 'list'>
<class 'str'>


### Stanza

In [9]:
import stanza
from stanza.utils.resources import DEFAULT_MODEL_DIR
print(DEFAULT_MODEL_DIR)

C:\Users\user\stanza_resources


In [10]:
en_stanza_nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma,pos', use_gpu=False)
zh_stanza_nlp = stanza.Pipeline(lang='zh-hant', processors='tokenize,lemma,pos', use_gpu=False)

2020-04-12 07:16:21 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |

2020-04-12 07:16:21 INFO: Use device: cpu
2020-04-12 07:16:21 INFO: Loading: tokenize
2020-04-12 07:16:21 INFO: Loading: pos
2020-04-12 07:16:22 INFO: Loading: lemma
2020-04-12 07:16:22 INFO: Done loading processors!
2020-04-12 07:16:22 INFO: Loading these models for language: zh-hant (Traditional_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

2020-04-12 07:16:22 INFO: Use device: cpu
2020-04-12 07:16:22 INFO: Loading: tokenize
2020-04-12 07:16:22 INFO: Loading: pos
2020-04-12 07:16:24 INFO: Loading: lemma
2020-04-12 07:16:24 INFO: Done loading processors!


In [6]:
zh_annotated_stanza_doc = zh_stanza_nlp(data)#stanza最慢

In [11]:
stanza = []
for i, sent in enumerate(zh_annotated_stanza_doc.sentences):
    for j, word in enumerate(sent.words):
        stanza.append(format(word.text))

### Ckiptagger

In [11]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from pathlib import Path
import os

In [12]:
path = str(Path.home())+'/ckip/'
if not os.path.exists(path):
    os.mkdir(path)
    data_utils.download_data_gdown(path)

In [13]:
zh_ws = WS(path+'/data/')
zh_pos = POS(path+'/data/')

In [ ]:
zh_annotated_ws = zh_ws(
    data_list,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary # words in this dictionary are encouraged
    # coerce_dictionary = dictionary2, # words in this dictionary are forced
)

In [14]:
ckip = []
for i, sentence in enumerate(zip(zh_annotated_ws)):
    for j, word in enumerate(zip(sentence[0])):
        ckip.append(word[0])

### Jieba

In [16]:
import os
from pathlib import Path
from ckiptagger import WS,POS,NER
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import urllib.request
import zipfile
import pandas as pd
from sklearn.metrics.classification import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import jieba
import jieba.posseg as pseg
URL = "http://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
if not os.path.exists(os.path.join(os.getcwd(),"dict.txt.big")):
    urllib.request.urlretrieve(URL,"dict.txt.big")
jieba.set_dictionary("dict.txt.big")

In [17]:
jieba.set_dictionary('dict.txt.big')
jieba.load_userdict("my.dict.txt")
print(DEFAULT_MODEL_DIR)

Building prefix dict from D:\jupyter\dict.txt.big ...
2020-04-11 21:30:25 DEBUG: Building prefix dict from D:\jupyter\dict.txt.big ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.uc7e69de6020c035eaa31e022ef353dfe.cache
2020-04-11 21:30:25 DEBUG: Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.uc7e69de6020c035eaa31e022ef353dfe.cache
Loading model cost 1.456 seconds.
2020-04-11 21:30:26 DEBUG: Loading model cost 1.456 seconds.
Prefix dict has been built succesfully.
2020-04-11 21:30:26 DEBUG: Prefix dict has been built succesfully.


C:\Users\user\stanza_resources


In [18]:
zh_annotated_jieba = list(jieba.cut(data, cut_all=False))#jieba跑好快

In [19]:
jieba = []
for j, word in enumerate(zh_annotated_jieba):
    jieba.append(word)
print(len(jieba))    

1509058


# 建字典

In [72]:
merge_st_ck = list(set(stanza) | set(ckip))#stanza or ckip
st_without_ck = list(set(stanza) - set(ckip))#stanza - ckip
ck_without_st = list(set(ckip) - set(stanza))#ckip - stanza
ji_without_st = list(set(jieba) - set(stanza))#jieba - stanza
ji_without_ck = list(set(jieba) - set(ckip))#jieba - ckip
dic_or = list(set(ji_without_st) | set(ji_without_ck))
a = list(set(ji_without_ck) - set(ji_without_st))
b = list(set(ji_without_st) - set(ji_without_ck))
dic_and = list(set(dic_or) - set(a) - set(b))
print(len(stanza))
print(len(ckip))
print(len(jieba))
print(len(dic_or))#聯集
print(len(dic_and))#交集

1401926
1351120
1509058
31470
15487


In [ ]:
# 匯出list
with open("dic.txt", 'w', encoding="utf-8") as f:
    for i in dic_and:
        f.write(i)
        f.write(',')

將15487筆斷詞結果匯出，人工篩選出3361筆，其中多包含政府機關名稱、成語，另外我們再加了台灣前百Youtuber的名單進去。

In [48]:
# 匯入
import sys
result = []
with open('06170114_userdefine.txt','r') as f:
    for line in f:
        item = line.strip('\n').split(',')
        result.append(item)
FINAL_dic = result[0]
len(FINAL_dic)

3361

# 優化斷詞結果

### Stanza

In [14]:
title_stanza_segment = []
for i in range(len(papers)):
    zh_annotated_stanza_doc = zh_stanza_nlp(papers[i]["title"])
    stanza = []
    for i, sent in enumerate(zh_annotated_stanza_doc.sentences):
        for j, word in enumerate(sent.words):
            stanza.append(format(word.text))
    title_stanza_segment.append(stanza)    

In [15]:
for i in range(len(title_stanza_segment)):    
    papers[i]["title_stanza_segment"] = title_stanza_segment[i]

In [16]:
content_stanza_segment = []
for i in range(len(papers)):
    zh_annotated_stanza_doc = zh_stanza_nlp(papers[i]["content"])
    stanza = []
    for i, sent in enumerate(zh_annotated_stanza_doc.sentences):
        for j, word in enumerate(sent.words):
            stanza.append(format(word.text))
    content_stanza_segment.append(stanza) 

In [17]:
for i in range(len(content_stanza_segment)):    
    papers[i]["content_stanza_segment"] = content_stanza_segment[i]

### Ckiptagger

In [21]:
#list to dict
word_to_weight = { i : 1 for i in FINAL_dic }

In [22]:
dictionary = construct_dictionary(word_to_weight)

In [42]:
for i in range(len(papers)):
    zh_annotated_ws = zh_ws([papers[i]["title"]], recommend_dictionary = dictionary)
    dic={"title_ckiptagger_segment" : zh_annotated_ws[0]}
    papers[i].update(dic)

In [43]:
for i in range(len(papers)):
    zh_annotated_ws = zh_ws([papers[i]["content"]], recommend_dictionary = dictionary)
    dic={"content_ckiptagger_segment" : zh_annotated_ws[0]}
    papers[i].update(dic)

In [47]:
# converting a list of dictionaries to json
with open('06170114_cts_0301_0403_news.json', 'w') as fout:
    json.dump(backup_papers , fout)